# Siamese model with numeric input

In [1]:
import sys
import os
sys.path.append(os.path.abspath("../src"))

In [2]:
import torch
from torch.utils.data import DataLoader, random_split
from protein_dataset import ProteinDataset
from encoders.numeric_protein_encoder import NumericProteinEncoder
from utils import load_numpy_dataset
from models.convolutive_train_pipeline import train_model

# Config
device = torch.device("cuda")
batch_size = 128
num_epochs = 5
lr = 1e-3

max_length = 1024

vocab_size = (NumericProteinEncoder().vocabulary_size() + 1) # +1 for padding token

# Dataset
X1, X2, y = load_numpy_dataset(f"../processed_data/numeric_chunks_{max_length}")
dataset = ProteinDataset(X1, X2, y)
train_size = int(0.7 * len(dataset))
val_size   = int(0.15 * len(dataset))
test_size  = len(dataset) - train_size - val_size
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader   = DataLoader(val_dataset, batch_size=batch_size)
test_loader  = DataLoader(test_dataset, batch_size=batch_size)

model = train_model(
    train_loader=train_loader,
    val_loader=val_loader,
    model_save_path="../models",
    embedding_dim=128,
    hidden_dim=256,
    vocab_size=vocab_size,
    lr=1e-3,
    epochs=50,
    patience=6
)


Epoch 1/50


Train loss: 0.4879


Val loss: 0.4352 | Accuracy: 0.8004
-- Best model saved.

Epoch 2/50


Train loss: 0.4337


Val loss: 0.4474 | Accuracy: 0.7926

Epoch 3/50


...

Val loss: 0.3522 | Accuracy: 0.8459

Epoch 49/50


Train loss: 0.3354


Val loss: 0.3531 | Accuracy: 0.8450

Epoch 50/50


Train loss: 0.3349


Val loss: 0.3674 | Accuracy: 0.8364


In [3]:
model.eval()
correct, total = 0, 0
with torch.no_grad():
    for x1, x2, y in test_loader:
        x1, x2, y = x1.to(device), x2.to(device), y.to(device)
        logits = model(x1, x2)
        preds = torch.sigmoid(logits) > 0.5
        correct += (preds == y.bool()).sum().item()
        total += y.size(0)

test_acc = correct / total
print(f"Test Accuracy: {test_acc:.4f}")

Test Accuracy: 0.8472
